<a id='report-mutable-scopes'></a>
<p style="font-weight:bold;"> <span style="font-size: 36px"> Report Mutable Scopes</span> </p>

This notebook contains the set up of mutable scopes used to achieve high interactivity with reports.

# References
Libraries and other notebooks which are needed for this notebook are imported below.

In [0]:
#!import "ReportScopes"

# Form Entity Scopes

## Helper Scopes

In [0]:
public interface MutableScopeWithWorkspace : IMutableScopeWithStorage<ReportStorage> {
    protected IWorkspace workspace => GetStorage().Workspace;   
}

## Currency Type

In [0]:
public interface CurrencyFormsEntity : IMutableScope {
    [DropdownEnum(typeof(CurrencyType))]
    CurrencyType CurrencyType { get; set; } 
}

## Scenario

In [0]:
public interface ScenarioFormsEntity : MutableScopeWithWorkspace {
    [DropdownMethod(nameof(GetScenarioAutocompleteAsync))]
    [Display(Name = "Scenario")]
    string ScenarioControl { get; set; }

    [NotVisible] IDictionary<string, string> ScenarioMapping { get; set; }
    protected string Scenario => !string.IsNullOrWhiteSpace(ScenarioControl) && ScenarioMapping is not null && ScenarioMapping.TryGetValue(ScenarioControl, out var value) ? value : null;
    
    async Task<IReadOnlyCollection<string>> GetScenarioAutocompleteAsync(string userInput, int page, int pageSize) {
        (ScenarioMapping, var orderedDropDownValues) = await workspace.GetAutocompleteMappings<Scenario>(true);
        return orderedDropDownValues.Where(x => userInput == null || x.Contains(userInput, StringComparison.OrdinalIgnoreCase)).ToArray(); 
    } 
}

## Reporting Node

In [0]:
[InitializeScope(nameof(InitReportingNode))]
public interface ReportingNodeFormsEntity : MutableScopeWithWorkspace {
    [DropdownMethod(nameof(GetReportingNodeAutocompleteAsync))]
    [Display(Name = "ReportingNode")]
    string ReportingNodeControl { get; set; }

    [NotVisible] IDictionary<string, string>  ReportingNodeMapping { get; set; }
    protected string ReportingNode => !string.IsNullOrWhiteSpace(ReportingNodeControl) && ReportingNodeMapping is not null && ReportingNodeMapping.TryGetValue( ReportingNodeControl, out var value)
        ? value
        : GetStorage().InitialReportingNode.SystemName; // Maybe these cases can be more specific

    async Task<IReadOnlyCollection<string>> GetReportingNodeAutocompleteAsync(string userInput, int page, int pageSize) {
        (ReportingNodeMapping, var orderedDropDownValues) = await workspace.GetAutocompleteMappings<ReportingNode>();
        return orderedDropDownValues.Where(x => userInput == null || x.Contains(userInput, StringComparison.OrdinalIgnoreCase)).ToArray(); 
    }

    async void InitReportingNode() {
        ReportingNodeControl = ParseDimensionToDisplayString(GetStorage().InitialReportingNode.SystemName, GetStorage().InitialReportingNode.DisplayName);
    }
}

## Reporting Period

### Monthly Period

In [0]:
[InitializeScope(nameof(InitReportingPeriod))]
public interface MonthlyPeriodFormsEntity : MutableScopeWithWorkspace {
    [DropdownMethod(nameof(GetReportingPeriodAutocompleteAsync))]
    string ReportingPeriod { get; set; }

    private char separator => 'M';
    private string[] ReportingPeriodSplit => ReportingPeriod.Split(separator);
    private int ParseReportingPeriod(int index) => !string.IsNullOrWhiteSpace(ReportingPeriod) && ReportingPeriodSplit is not null && Int32.TryParse(ReportingPeriodSplit.ElementAtOrDefault(index), out int value)
        ? value
        : default;

    protected int Year => ParseReportingPeriod(0);
    protected int Month => ParseReportingPeriod(1);

    async Task<IReadOnlyCollection<string>> GetReportingPeriodAutocompleteAsync(string userInput, int page, int pageSize) => 
        await workspace.Query<PartitionByReportingNodeAndPeriod>()
            .Where(x => x.Scenario == null)
            .OrderByDescending(x => x.Year)
            .ThenByDescending(x => x.Month)
            .Select(x => ParseReportingPeriodToDisplayString(x.Year, x.Month, separator))
            .Where(x => userInput == null || x.Contains(userInput, StringComparison.OrdinalIgnoreCase))
            .ToArrayAsync();

    async void InitReportingPeriod() {
        ReportingPeriod = ParseReportingPeriodToDisplayString(GetStorage().InitialReportingPeriod.Year, GetStorage().InitialReportingPeriod.Month, separator);
    }
}

## Filters

In [0]:
public interface BasicFilterFormsEntity : MutableScopeWithWorkspace {
    [DropdownValues("", "GroupOfContract", "Novelty", "EconomicBasis")]
    string FilterName { get; set; }

    [DropdownMethod(nameof(GetBasicFilterAsync))]
    string FilterValue { get; set; }

    [DropdownValues("", "Add", "Remove")]
    string FilterAction { get; set; }
    
    async Task<IReadOnlyCollection<string>> GetBasicFilterAsync(string userInput, int page, int pageSize) =>
        new string[]{ null }.Concat(
            await (FilterName switch
            {
                "GroupOfContract" => workspace.Query<DataNode>().Select(x => x.SystemName),
                "Novelty" => workspace.Query<Novelty>().Select(x => x.SystemName),
                "EconomicBasis" => workspace.Query<EconomicBasis>().Select(x => x.SystemName),
                _ => Enumerable.Empty<string>().AsQueryable()
            }) .Where(x => userInput == null || x.Contains(userInput, StringComparison.OrdinalIgnoreCase)).OrderBy(x => x).ToArrayAsync()).ToArray();
        
    [NotVisible] IReadOnlyCollection<(string filterName, string filterValue)> InputDataFilter { get; set; }
    
    // This is just a cast... do we need this?
    protected (string filterName, object filterValue)[] dataFilter => (InputDataFilter is null ? Enumerable.Empty<(string, object)>() : InputDataFilter.Select(x => (x.filterName, (object)x.filterValue))).ToArray();

    IReadOnlyCollection<(string filterName, string filterValue)> GetFilters()
    {
        if(FilterAction == "Add")
            AddFilter(FilterName, FilterValue);
        if(FilterAction == "Remove")
            RemoveFilter(FilterName, FilterValue);
        return InputDataFilter;
    }

    private void AddFilter(string filterName, string filterValue)
    {
        if(InputDataFilter == null)
            InputDataFilter = Enumerable.Empty<(string, string)>().ToArray();
        if(!InputDataFilter.Contains((filterName, filterValue)))
            InputDataFilter = InputDataFilter.Append((filterName, filterValue)).ToArray();
    }
    
    private void RemoveFilter(string filterName, string filterValue)
    {
        if(InputDataFilter != null && InputDataFilter.Contains((filterName, filterValue)))
        {   var f = InputDataFilter.ToList();
            f.Remove((filterName, filterValue));
            InputDataFilter = f.ToArray();
        }
    }
}

## Slice and Dice

In [0]:
public interface BasicSliceAndDiceFormsEntity : MutableScopeWithWorkspace {
    [DropdownValues("", "GroupOfContract")]
    string SliceRowName { get; set; }

    [DropdownMethod(nameof(GetSliceColumnNameAutocomplete))]
    string SliceColumnName { get; set; }

    protected IReadOnlyCollection<string> InputRowSlices => (SliceRowName is null ? Enumerable.Empty<string>() : SliceRowName.RepeatOnce()).ToArray();
    [NotVisible] IReadOnlyCollection<string> defaultRowSlices { get; set; }
    protected string[] rowSlices => defaultRowSlices.Union(InputRowSlices).ToArray();

    protected IReadOnlyCollection<string> InputColumnSlices => (SliceColumnName is null ? Enumerable.Empty<string>() : SliceColumnName.RepeatOnce()).ToArray();
    [NotVisible] IReadOnlyCollection<string> defaultColumnSlices { get; set; }
    protected string[] columnSlices => defaultColumnSlices.Union(InputColumnSlices).ToArray();

    IReadOnlyCollection<string> GetSliceColumnNameAutocomplete() => new [] {"", "GroupOfContract", "AmountType"};
}

# Report Scopes

In [0]:
[InitializeScope(nameof(InitReportStorageScopeAsync))]
public interface ReportScope : IMutableScope<string>, ReportingNodeFormsEntity, MonthlyPeriodFormsEntity, ScenarioFormsEntity, CurrencyFormsEntity, BasicSliceAndDiceFormsEntity, BasicFilterFormsEntity {
    protected Systemorph.Vertex.Pivot.Builder.Interfaces.IPivotFactory report => GetStorage().Report;
    protected int headerColumnWidthValue => 250;

    HashSet<(ReportIdentity, CurrencyType)> GetDataIdentities() => GetStorage().GetIdentities((Year, Month), ReportingNode, Scenario, CurrencyType); // TODO, add filter for identities, if the property is exposed at this level
       
    async Task<GridOptions> GetReportTaskAsync(IDataCube<ReportVariable> data) {
        return await report.ForDataCube(data)
            .WithQuerySource(workspace)
            .SliceRowsBy(rowSlices)
            .SliceColumnsBy(columnSlices)
            .ReportGridOptions(headerColumnWidth: headerColumnWidthValue)
            .ExecuteAsync();
    }

    async Task InitReportStorageScopeAsync() { // This has the Async issue, but imo it should come in the future
       await GetStorage().InitializeReportIndependentCacheAsync();
    }
}

In [0]:
public interface Data : IMutableScope<((int year, int month) reportingPeriod, string reportingNode, string scenario, CurrencyType currencyType, (string filterName, object filterValue)[] dataFilter)> {
    IDataCube<ReportVariable> InputDataCube { get; set; }

    IDataCube<ReportVariable> DataCube { get {
        if(InputDataCube is null) return Enumerable.Empty<ReportVariable>().ToDataCube();
        var filteredDataCube = (Identity.dataFilter is null || Identity.dataFilter.Length == 0) ? InputDataCube : InputDataCube.Filter(Identity.dataFilter); 
        if(Identity.scenario != "Delta") return filteredDataCube;
        var bestEstimateById = filteredDataCube.Where(x => x.Scenario == null).ToDictionary(x => x.ToIdentityString());
        return filteredDataCube.Select(x => x.Scenario == null ? x : x with { Value = x.Value - (bestEstimateById.TryGetValue((x with {Scenario = null}).ToIdentityString(), out var be)? be.Value : 0d) }).ToDataCube();
    }}
} 

# Report Scopes

## Best Estimate PV

In [0]:
[InitializeScope(nameof(Init))]
public interface PvReport : ReportScope {

    public IDataCube<ReportVariable> GetData() => GetScopes<LockedBestEstimate>(GetDataIdentities()).Aggregate().LockedBestEstimate + GetScopes<CurrentBestEstimate>(GetDataIdentities()).Aggregate().CurrentBestEstimate;

    async Task<GridOptions> ToReportAsync() {
        await GetStorage().InitializeAsync((Year, Month), ReportingNode, Scenario, CurrencyType);
        var dataScope = GetScope<Data>(((Year, Month), ReportingNode, Scenario, CurrencyType, dataFilter));
        dataScope.InputDataCube = GetData();
        return await GetReportTaskAsync(dataScope.DataCube);
    }

     void Init() {
         // BasicSliceAndDiceFormsEntity
         defaultRowSlices = new string[] { "Novelty", "VariableType" };
         defaultColumnSlices = new string[] { "Currency", "LiabilityType", "EconomicBasis" };
    }
}

## Risk Adjustment PV

# IFRS 17 Reports

This class is used to trigger the calculation of the reports and it is exposed to the end-user in the reports.

In [0]:
public class Ifrs17Interactive 
{
    //private Systemorph.Vertex.Scopes.Proxy.IScopeFactory scopes;
    private Systemorph.Vertex.Pivot.Builder.Interfaces.IPivotFactory report;
    private Systemorph.InteractiveObjects.InteractiveObjectVariable interactiveObject;
    private ReportStorage storage;
    
    //reset
    public void Reset(IWorkspace workspace) => storage = new ReportStorage(workspace, report);

    //constructor
    public Ifrs17Interactive (IWorkspace workspace, Systemorph.Vertex.Pivot.Builder.Interfaces.IPivotFactory report, Systemorph.InteractiveObjects.InteractiveObjectVariable interactiveObject)
    {
        //this.scopes = scopes; 
        this.report = report;
        this.interactiveObject = interactiveObject;
        storage = new ReportStorage(workspace, report);
    }
     
    public Systemorph.Vertex.InteractiveObjects.InteractiveObjectView PvReportFormsEntity =>
        interactiveObject.CreateView(nameof(PvReportFormsEntity),
            state => state.GetScope<PvReport>(nameof(PvReport), o => o.WithStorage(storage)));
    
    public Systemorph.Vertex.InteractiveObjects.InteractiveObjectView PvReportGrid =>
        interactiveObject.CreateView(nameof(PvReportGrid),
            state => {
                var scope = state.GetScope<PvReport>(nameof(PvReport), o => o.WithStorage(storage));
                var filters = scope.GetFilters(); // Not used and should be improved
                return scope.ToReportAsync();
            });


    //public IIfrs17Report PresentValues => reportUniverse.GetScope<PvReport>();
    // public IIfrs17Report RiskAdjustments => reportUniverse.GetScope<RaReport>();
    // public IIfrs17Report WrittenActuals => reportUniverse.GetScope<WrittenReport>();
    // public IIfrs17Report AccrualActuals => reportUniverse.GetScope<AccrualReport>();
    // public IIfrs17Report DeferralActuals => reportUniverse.GetScope<DeferralReport>();
    // public IIfrs17Report FulfillmentCashflows => reportUniverse.GetScope<FcfReport>();
    // public IIfrs17Report ExperienceAdjustments => reportUniverse.GetScope<ExpAdjReport>();
    // public IIfrs17Report TechnicalMargins => reportUniverse.GetScope<TmReport>();
    // public IIfrs17Report AllocatedTechnicalMargins => reportUniverse.GetScope<CsmReport>();
    // public IIfrs17Report ActuarialLrc => reportUniverse.GetScope<ActLrcReport>();
    // public IIfrs17Report Lrc => reportUniverse.GetScope<LrcReport>();
    // public IIfrs17Report ActuarialLic => reportUniverse.GetScope<ActLicReport>();
    // public IIfrs17Report Lic => reportUniverse.GetScope<LicReport>();
    // public IIfrs17Report FinancialPerformance => reportUniverse.GetScope<FpReport>();
}